In [2]:
from eptr2 import EPTR2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
start_date = pd.to_datetime("2023-04-25")
end_date = pd.to_datetime("2024-04-23")

In [4]:
def convert_to_dataframe(data):
    items = data['items']
    
    df = pd.DataFrame(items)
    
    df['date'] = pd.to_datetime(df['date'])
    
    return df

In [5]:
eptr = EPTR2(
    ssl_verify=True,
    postprocess=False,  
    get_raw_response=False
)

In [6]:
day_ahead_prices = exchange = eptr.call("mcp", start_date=start_date, end_date=end_date)
day_ahead_prices = convert_to_dataframe(day_ahead_prices)
day_ahead_prices = day_ahead_prices.drop(columns=['priceEur'])
day_ahead_prices = day_ahead_prices.drop(columns=['priceUsd'])
day_ahead_prices['date'] = day_ahead_prices['date'].dt.tz_localize(None)
day_ahead_prices['date'] = pd.to_datetime(day_ahead_prices['date'])
day_ahead_prices

,date,hour,price
0,2023-04-25 00:00:00,00:00,2600.00
1,2023-04-25 01:00:00,01:00,2600.00
2,2023-04-25 02:00:00,02:00,2235.58
3,2023-04-25 03:00:00,03:00,1700.00
4,2023-04-25 04:00:00,04:00,1700.00
...,...,...,...
8755,2024-04-23 19:00:00,19:00,2700.00
8756,2024-04-23 20:00:00,20:00,2700.00
8757,2024-04-23 21:00:00,21:00,2700.00
8758,2024-04-23 22:00:00,22:00,2366.89


In [7]:
pd.set_option('display.width', 300)
print(day_ahead_prices.head(24))

                  date   hour    price
0  2023-04-25 00:00:00  00:00  2600.00
1  2023-04-25 01:00:00  01:00  2600.00
2  2023-04-25 02:00:00  02:00  2235.58
3  2023-04-25 03:00:00  03:00  1700.00
4  2023-04-25 04:00:00  04:00  1700.00
5  2023-04-25 05:00:00  05:00  2159.99
6  2023-04-25 06:00:00  06:00  2235.58
7  2023-04-25 07:00:00  07:00  1525.00
8  2023-04-25 08:00:00  08:00  2600.00
9  2023-04-25 09:00:00  09:00  2599.98
10 2023-04-25 10:00:00  10:00  2099.99
11 2023-04-25 11:00:00  11:00  2100.00
12 2023-04-25 12:00:00  12:00  1045.00
13 2023-04-25 13:00:00  13:00  1698.97
14 2023-04-25 14:00:00  14:00  1700.00
15 2023-04-25 15:00:00  15:00  1698.96
16 2023-04-25 16:00:00  16:00  2000.00
17 2023-04-25 17:00:00  17:00  2300.01
18 2023-04-25 18:00:00  18:00  2600.00
19 2023-04-25 19:00:00  19:00  2600.00
20 2023-04-25 20:00:00  20:00  2600.00
21 2023-04-25 21:00:00  21:00  2600.00
22 2023-04-25 22:00:00  22:00  1900.00
23 2023-04-25 23:00:00  23:00  1351.43


### intra day

In [8]:
def calculate_quantiles(df):
    grouped = df.groupby(['date', 'hour'])
    
    results = []
    
    for (date, hour), group in grouped:
        group = group.sort_values(by='price')
        group['cumulative_quantity'] = group['quantity'].cumsum()
        
        total_quantity = group['cumulative_quantity'].iloc[-1]
        
        target_quantiles = [0.25, 0.5, 0.75]
        targets = [total_quantity * q for q in target_quantiles]
        
        for quantile, target in zip(target_quantiles, targets):
            matched_row = group[group['cumulative_quantity'] >= target].iloc[0]
            price_at_quantile = matched_row['price']
            cumulative_lots_at_quantile = total_quantity - matched_row['cumulative_quantity']
            
            results.append({
                'date': date,
                'hour': hour,
                'price': price_at_quantile,
                'quantile': quantile,
                'cumulative_lots': cumulative_lots_at_quantile
            })
    
    return pd.DataFrame(results)

In [9]:
all_intra_day_data = pd.read_csv("all_intraday_data.csv")
all_intra_day_data['date'] = pd.to_datetime(all_intra_day_data['date'])
all_intra_day_data = all_intra_day_data.drop('contract_date', axis=1)
all_intra_day_data = all_intra_day_data.drop('contract_hour', axis=1)
all_intra_day_data = all_intra_day_data.drop('id', axis=1)

In [10]:
all_quantiles_data = calculate_quantiles(all_intra_day_data)

In [11]:
all_quantiles_data  = all_quantiles_data[(all_quantiles_data["date"] > "2023-04-24")&(all_quantiles_data["date"] < "2024-04-24")]
all_quantiles_data

,date,hour,price,quantile,cumulative_lots
59763,2023-04-25,00:00,2522.97,0.25,27084
59764,2023-04-25,00:00,2535.00,0.50,18116
59765,2023-04-25,00:00,2580.00,0.75,8691
59766,2023-04-25,01:00,2522.00,0.25,19992
59767,2023-04-25,01:00,2529.26,0.50,13287
...,...,...,...,...,...
86026,2024-04-23,22:00,2450.00,0.50,10505
86027,2024-04-23,22:00,2595.70,0.75,5249
86028,2024-04-23,23:00,2014.90,0.25,19889
86029,2024-04-23,23:00,2306.00,0.50,13252


### Production

In [12]:
production =pd.read_csv('AlgopolyPerformance-PerformanceReport_2024-04-25_06-16-54.csv')

production = production[(production['time'] >= "2023-04-25") & (production['time'] < "2024-04-24")]
production = production[(production['eic'] == '40W000000010903W')]

production = production[['time', 'forecast']]

production['time'] = pd.to_datetime(production['time'])

production = production.sort_values(by=['time', 'forecast'], ascending=[True, False])

production = production.drop_duplicates(subset=['time'], keep='first')

production = production.rename(columns={'forecast': 'predicted_production'})

production.rename(columns={'time': 'date'}, inplace=True)

production['date'] = production['date'].dt.tz_localize(None)

production

,date,predicted_production
454702,2023-04-25 00:00:00,5.460000
749147,2023-04-25 01:00:00,7.060000
64952,2023-04-25 02:00:00,7.720000
584674,2023-04-25 03:00:00,9.060000
108499,2023-04-25 04:00:00,14.025820
...,...,...
260510,2024-04-23 19:00:00,51.530000
357861,2024-04-23 20:00:00,53.850000
228946,2024-04-23 21:00:00,58.062974
554018,2024-04-23 22:00:00,53.740000


In [13]:
production['day'] = production['date'].dt.strftime('%Y-%m-%d')
production['hour'] = production['date'].dt.strftime('%H:00')
production

,date,predicted_production,day,hour
454702,2023-04-25 00:00:00,5.460000,2023-04-25,00:00
749147,2023-04-25 01:00:00,7.060000,2023-04-25,01:00
64952,2023-04-25 02:00:00,7.720000,2023-04-25,02:00
584674,2023-04-25 03:00:00,9.060000,2023-04-25,03:00
108499,2023-04-25 04:00:00,14.025820,2023-04-25,04:00
...,...,...,...,...
260510,2024-04-23 19:00:00,51.530000,2024-04-23,19:00
357861,2024-04-23 20:00:00,53.850000,2024-04-23,20:00
228946,2024-04-23 21:00:00,58.062974,2024-04-23,21:00
554018,2024-04-23 22:00:00,53.740000,2024-04-23,22:00


## Solving

In [14]:
import pulp as pl
import pandas as pd

def setup_and_solve_trading_model(all_quantiles_data, production_wind, dayahead):
    """
    Set up and solve the linear programming problem for electricity trading optimization.

    Parameters:
    all_quantiles_data (pd.DataFrame): DataFrame containing hourly quantiles data with 'Hour', 'Quantile', 'Price', and 'Cumulative Lots'.
    production_wind (pd.DataFrame): DataFrame containing production data from wind with columns 'hour' and 'forecast'.
    dayahead (pd.DataFrame): DataFrame containing day-ahead prices with columns 'hour' and 'price'.

    Returns:
    dict: Dictionary containing the solution variables and the objective value.
    """
    # Ensure that dataframes use a consistent 'hour' format, assuming no change needed if already "HH:00"
    # No conversion required for 'dayahead' as it's already "HH:00"

    # Create dictionaries for model parameters
    intraday_dict = all_quantiles_data.set_index(['hour', 'quantile']).to_dict('index')
    production_wind_dict = production_wind.set_index('hour')['forecast'].to_dict()
    dayahead_dict = dayahead.set_index('hour')['price'].to_dict()

    # Set up the model
    model = pl.LpProblem("Electricity_Trading", pl.LpMaximize)

    # Time periods and quantiles
    T = dayahead['hour'].unique()  # Assuming hours are already in 'HH:00' format
    Q = all_quantiles_data['quantile'].unique()

    # Decision Variables
    Q1 = pl.LpVariable.dicts("Q1", T, lowBound=0)
    Q2 = pl.LpVariable.dicts("Q2", [(t, q) for t in T for q in Q], lowBound=0)

    # Objective Function
    model += pl.lpSum([dayahead_dict[t] * Q1[t] for t in T if t in dayahead_dict] +
                      [intraday_dict[(t, q)]['price'] * Q2[(t, q)] for t in T for q in Q if (t, q) in intraday_dict])

    # Constraints
    for t in T:
        if t in production_wind_dict:
            model += Q1[t] + pl.lpSum([Q2[(t, q)] for q in Q if (t, q) in intraday_dict]) <= production_wind_dict[t], f"Production_{t}"

        for q in Q:
            if (t, q) in intraday_dict:
                model += Q2[(t, q)] <= intraday_dict[(t, q)]['cumulative_lots'], f"Demand_{t}_{q}"

    # Solve the model
    model.solve()

    if pl.LpStatus[model.status] != 'Optimal':
        print("No feasible solution found.")
        return None

    # Collect results
    results = {'Variables': {v.name: v.varValue for v in model.variables()},
               'Objective Value': pl.value(model.objective)}

    return results


In [15]:
unique_days = day_ahead_prices['date'].dt.date.unique()

In [16]:
import pandas as pd
from datetime import datetime, timedelta

daily_results = []

results_list = []

for date in unique_days:

    day_df = day_ahead_prices[day_ahead_prices["date"].dt.date == date]
    day_df['price'].fillna(0, inplace=True)

    intraday_data = all_quantiles_data[all_quantiles_data["date"].dt.date == date]

    production_data = production[production["date"].dt.date == date]

    production_wind = pd.DataFrame({'hour': production_data['hour'], 'forecast': production_data['predicted_production']})
    production_wind['forecast'].fillna(0, inplace=True)

    dayahead = pd.DataFrame({'hour': day_df['hour'], 'price': day_df['price']})
    intraday_data = pd.DataFrame({'hour': intraday_data['hour'], 'quantile' : intraday_data['quantile'], 'price': intraday_data['price'], 'cumulative_lots': intraday_data['cumulative_lots']})
    
    result = setup_and_solve_trading_model(intraday_data[['hour', 'quantile', 'price', 'cumulative_lots']],
                                                production_wind, dayahead)
    
    results_list.append({'date': date, 'results': result})
    daily_results.append(result)


results_df = pd.DataFrame(results_list)
results_df

C:\Users\Administrator\AppData\Local\Temp\ipykernel_5248\2137799305.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_df['price'].fillna(0, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_5248\2137799305.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_df['price'].fillna(0, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_5248\2137799305.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

,date,results
0,2023-04-25,"{'Variables': {'Q1_00:00': 5.46, 'Q1_01:00': 7..."
1,2023-04-26,"{'Variables': {'Q1_00:00': 0.0, 'Q1_01:00': 0...."
2,2023-04-27,"{'Variables': {'Q1_00:00': 0.0, 'Q1_01:00': 0...."
3,2023-04-28,"{'Variables': {'Q1_00:00': 62.391755, 'Q1_01:0..."
4,2023-04-29,"{'Variables': {'Q1_00:00': 41.021926, 'Q1_01:0..."
...,...,...
360,2024-04-19,"{'Variables': {'Q1_00:00': 20.85, 'Q1_01:00': ..."
361,2024-04-20,"{'Variables': {'Q1_00:00': 0.0, 'Q1_01:00': 0...."
362,2024-04-21,"{'Variables': {'Q1_00:00': 0.0, 'Q1_01:00': 0...."
363,2024-04-22,"{'Variables': {'Q1_00:00': 0.0, 'Q1_01:00': 0...."


In [17]:
structured_results = []
current_date = pd.to_datetime("2023-04-25")

for result in daily_results:
    day_dict = {'Date': current_date.strftime('%Y-%m-%d')}
    day_dict.update(result['Variables'])
    day_dict['Objective Value'] = result['Objective Value']
    structured_results.append(day_dict)
    current_date += timedelta(days=1)

results_df = pd.DataFrame(structured_results)

expected_total_revenue = results_df['Objective Value'].sum()

In [18]:
expected_total_revenue

455702712.64942944

In [19]:
results_df.to_csv('trading_optimization_results_day_before_expected.csv', index=False)


In [20]:
results_df

,Date,Q1_00:00,Q1_01:00,Q1_02:00,Q1_03:00,Q1_04:00,Q1_05:00,Q1_06:00,Q1_07:00,Q1_08:00,...,"Q2_('21:00',_0.25)","Q2_('21:00',_0.5)","Q2_('21:00',_0.75)","Q2_('22:00',_0.25)","Q2_('22:00',_0.5)","Q2_('22:00',_0.75)","Q2_('23:00',_0.25)","Q2_('23:00',_0.5)","Q2_('23:00',_0.75)",Objective Value
0,2023-04-25,5.460000,7.060000,7.720000,9.060000,14.025820,13.950000,14.630000,15.630000,13.170000,...,0.0,0.0,30.650000,0.0,0.0,30.030000,0.0,0.0,30.270000,1.116303e+06
1,2023-04-26,0.000000,0.000000,0.000000,0.000000,28.020000,30.936151,0.000000,0.000000,0.000000,...,0.0,0.0,16.310000,0.0,0.0,12.750000,0.0,0.0,13.110000,1.595606e+06
2,2023-04-27,0.000000,0.000000,0.000000,2.795037,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,65.484879,0.0,0.0,65.387576,0.0,0.0,64.222108,1.844614e+06
3,2023-04-28,62.391755,59.919545,60.571263,59.798947,61.185766,61.828438,60.784790,62.487898,0.000000,...,0.0,0.0,60.546332,0.0,0.0,0.000000,0.0,0.0,0.000000,3.292749e+06
4,2023-04-29,41.021926,0.000000,0.000000,0.000000,31.990000,0.000000,36.961611,0.000000,39.578429,...,0.0,0.0,35.735778,0.0,0.0,32.514035,0.0,0.0,34.341344,2.279086e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2024-04-19,20.850000,20.448717,20.657458,17.520000,15.190000,0.000000,0.000000,9.610000,0.000000,...,0.0,0.0,12.590000,0.0,0.0,0.000000,0.0,0.0,0.000000,8.359322e+05
361,2024-04-20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,1.780000,0.0,0.0,1.460000,0.0,0.0,1.130000,7.375374e+05
362,2024-04-21,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.980000,0.000000,...,0.0,0.0,46.520000,0.0,0.0,47.660000,0.0,0.0,49.368974,8.585275e+05
363,2024-04-22,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,2.160000,0.0,0.0,3.070000,0.0,0.0,3.680000,6.959576e+05


In [21]:
import pandas as pd


final_data = []

for index, row in results_df.iterrows():
    date = row['Date']
    objective_value = row['Objective Value']
    
    for column in results_df.columns:
        if column.startswith('Q1') and column not in ['Date', 'Objective Value']:
            hour = column.split('_')[1]  # Extract hour from column name
            day_ahead_bidded = row[column]
            
            final_data.append({'date': date, 'hour': hour, 'day_ahead_bidded': day_ahead_bidded, 'Objective Value': objective_value})

final_dayahead_df = pd.DataFrame(final_data)

final_dayahead_df


,date,hour,day_ahead_bidded,Objective Value
0,2023-04-25,00:00,5.46000,1.116303e+06
1,2023-04-25,01:00,7.06000,1.116303e+06
2,2023-04-25,02:00,7.72000,1.116303e+06
3,2023-04-25,03:00,9.06000,1.116303e+06
4,2023-04-25,04:00,14.02582,1.116303e+06
...,...,...,...,...
8755,2024-04-23,19:00,0.00000,1.524395e+06
8756,2024-04-23,20:00,0.00000,1.524395e+06
8757,2024-04-23,21:00,0.00000,1.524395e+06
8758,2024-04-23,22:00,0.00000,1.524395e+06


## Real Day Ahead Check

In [22]:
start_date = pd.to_datetime("2023-04-25")
end_date = pd.to_datetime("2024-04-23")
real_dayahead_prices = eptr.call("mcp", start_date=start_date, end_date=end_date)
real_dayahead_prices = convert_to_dataframe(real_dayahead_prices)

In [23]:
real_dayahead_prices = real_dayahead_prices.drop(columns=['priceEur', 'priceUsd'])
real_dayahead_prices['date'] = pd.to_datetime(real_dayahead_prices['date']).dt.tz_localize(None)
real_dayahead_prices

,date,hour,price
0,2023-04-25 00:00:00,00:00,2600.00
1,2023-04-25 01:00:00,01:00,2600.00
2,2023-04-25 02:00:00,02:00,2235.58
3,2023-04-25 03:00:00,03:00,1700.00
4,2023-04-25 04:00:00,04:00,1700.00
...,...,...,...
8755,2024-04-23 19:00:00,19:00,2700.00
8756,2024-04-23 20:00:00,20:00,2700.00
8757,2024-04-23 21:00:00,21:00,2700.00
8758,2024-04-23 22:00:00,22:00,2366.89


In [24]:
final_dayahead_df['date'] = pd.to_datetime(final_dayahead_df['date'] + ' ' + final_dayahead_df['hour'])
real_dayahead_prices['date'] = pd.to_datetime(real_dayahead_prices['date'])

dayahead_df = pd.merge(real_dayahead_prices, final_dayahead_df, on="date")
dayahead_df

,date,hour_x,price,hour_y,day_ahead_bidded,Objective Value
0,2023-04-25 00:00:00,00:00,2600.00,00:00,5.46000,1.116303e+06
1,2023-04-25 01:00:00,01:00,2600.00,01:00,7.06000,1.116303e+06
2,2023-04-25 02:00:00,02:00,2235.58,02:00,7.72000,1.116303e+06
3,2023-04-25 03:00:00,03:00,1700.00,03:00,9.06000,1.116303e+06
4,2023-04-25 04:00:00,04:00,1700.00,04:00,14.02582,1.116303e+06
...,...,...,...,...,...,...
8755,2024-04-23 19:00:00,19:00,2700.00,19:00,0.00000,1.524395e+06
8756,2024-04-23 20:00:00,20:00,2700.00,20:00,0.00000,1.524395e+06
8757,2024-04-23 21:00:00,21:00,2700.00,21:00,0.00000,1.524395e+06
8758,2024-04-23 22:00:00,22:00,2366.89,22:00,0.00000,1.524395e+06


In [25]:
dayahead_df['revenue'] = dayahead_df['price'] * dayahead_df['day_ahead_bidded']
dayahead_df

,date,hour_x,price,hour_y,day_ahead_bidded,Objective Value,revenue
0,2023-04-25 00:00:00,00:00,2600.00,00:00,5.46000,1.116303e+06,14196.0000
1,2023-04-25 01:00:00,01:00,2600.00,01:00,7.06000,1.116303e+06,18356.0000
2,2023-04-25 02:00:00,02:00,2235.58,02:00,7.72000,1.116303e+06,17258.6776
3,2023-04-25 03:00:00,03:00,1700.00,03:00,9.06000,1.116303e+06,15402.0000
4,2023-04-25 04:00:00,04:00,1700.00,04:00,14.02582,1.116303e+06,23843.8940
...,...,...,...,...,...,...,...
8755,2024-04-23 19:00:00,19:00,2700.00,19:00,0.00000,1.524395e+06,0.0000
8756,2024-04-23 20:00:00,20:00,2700.00,20:00,0.00000,1.524395e+06,0.0000
8757,2024-04-23 21:00:00,21:00,2700.00,21:00,0.00000,1.524395e+06,0.0000
8758,2024-04-23 22:00:00,22:00,2366.89,22:00,0.00000,1.524395e+06,0.0000


## Real Intraday Check

In [26]:
import pandas as pd

final_data = []

for index, row in results_df.iterrows():
    date = row['Date']
    objective_value = row['Objective Value']
    
    for column in results_df.columns:
        if column.startswith('Q2') and column not in ['Date', 'Objective Value']:
            text = column.split('_')
            hour = text[1]  # Extract hour from column name
            quantile = text[-1]
            intraday_bidded = row[column]
            
            final_data.append({'date': date, 'hour': hour, 'quantile': quantile, 'intraday_bidded': intraday_bidded, 'Objective Value': objective_value})

final_intraday_df = pd.DataFrame(final_data)

final_intraday_df['hour'] = final_intraday_df['hour'].str.replace("(", "").str.replace(")", "").str.replace("'", "").str.replace(",", "")
final_intraday_df['quantile'] = final_intraday_df['quantile'].str.replace(")", "")

final_intraday_df

,date,hour,quantile,intraday_bidded,Objective Value
0,2023-04-25,00:00,0.25,0.00,1.116303e+06
1,2023-04-25,00:00,0.5,0.00,1.116303e+06
2,2023-04-25,00:00,0.75,0.00,1.116303e+06
3,2023-04-25,01:00,0.25,0.00,1.116303e+06
4,2023-04-25,01:00,0.5,0.00,1.116303e+06
...,...,...,...,...,...
26275,2024-04-23,22:00,0.5,0.00,1.524395e+06
26276,2024-04-23,22:00,0.75,53.74,1.524395e+06
26277,2024-04-23,23:00,0.25,0.00,1.524395e+06
26278,2024-04-23,23:00,0.5,0.00,1.524395e+06


In [27]:
final_intraday_df['date'] = pd.to_datetime(final_intraday_df['date'] + ' ' + final_intraday_df['hour'])
final_intraday_df

,date,hour,quantile,intraday_bidded,Objective Value
0,2023-04-25 00:00:00,00:00,0.25,0.00,1.116303e+06
1,2023-04-25 00:00:00,00:00,0.5,0.00,1.116303e+06
2,2023-04-25 00:00:00,00:00,0.75,0.00,1.116303e+06
3,2023-04-25 01:00:00,01:00,0.25,0.00,1.116303e+06
4,2023-04-25 01:00:00,01:00,0.5,0.00,1.116303e+06
...,...,...,...,...,...
26275,2024-04-23 22:00:00,22:00,0.5,0.00,1.524395e+06
26276,2024-04-23 22:00:00,22:00,0.75,53.74,1.524395e+06
26277,2024-04-23 23:00:00,23:00,0.25,0.00,1.524395e+06
26278,2024-04-23 23:00:00,23:00,0.5,0.00,1.524395e+06


In [28]:
all_quantiles_data['date'] = all_quantiles_data['date'].astype(str)
all_quantiles_data['date'] = pd.to_datetime(all_quantiles_data['date'] + ' ' + all_quantiles_data['hour'])
all_quantiles_data['date'] = pd.to_datetime(all_quantiles_data['date'])
all_quantiles_data

,date,hour,price,quantile,cumulative_lots
59763,2023-04-25 00:00:00,00:00,2522.97,0.25,27084
59764,2023-04-25 00:00:00,00:00,2535.00,0.50,18116
59765,2023-04-25 00:00:00,00:00,2580.00,0.75,8691
59766,2023-04-25 01:00:00,01:00,2522.00,0.25,19992
59767,2023-04-25 01:00:00,01:00,2529.26,0.50,13287
...,...,...,...,...,...
86026,2024-04-23 22:00:00,22:00,2450.00,0.50,10505
86027,2024-04-23 22:00:00,22:00,2595.70,0.75,5249
86028,2024-04-23 23:00:00,23:00,2014.90,0.25,19889
86029,2024-04-23 23:00:00,23:00,2306.00,0.50,13252


In [29]:
all_quantiles_data['quantile'] = all_quantiles_data['quantile'].astype(float)
final_intraday_df['quantile'] = final_intraday_df['quantile'].astype(float)

final_intraday_df = pd.merge(all_quantiles_data[["date","hour","price","quantile"]], final_intraday_df, on=['date', 'hour', 'quantile'], how='left')
final_intraday_df

,date,hour,price,quantile,intraday_bidded,Objective Value
0,2023-04-25 00:00:00,00:00,2522.97,0.25,0.00,1.116303e+06
1,2023-04-25 00:00:00,00:00,2535.00,0.50,0.00,1.116303e+06
2,2023-04-25 00:00:00,00:00,2580.00,0.75,0.00,1.116303e+06
3,2023-04-25 01:00:00,01:00,2522.00,0.25,0.00,1.116303e+06
4,2023-04-25 01:00:00,01:00,2529.26,0.50,0.00,1.116303e+06
...,...,...,...,...,...,...
26263,2024-04-23 22:00:00,22:00,2450.00,0.50,0.00,1.524395e+06
26264,2024-04-23 22:00:00,22:00,2595.70,0.75,53.74,1.524395e+06
26265,2024-04-23 23:00:00,23:00,2014.90,0.25,0.00,1.524395e+06
26266,2024-04-23 23:00:00,23:00,2306.00,0.50,0.00,1.524395e+06


In [30]:
real_intra_day_data = pd.read_csv("all_intraday_data.csv")
real_intra_day_data = real_intra_day_data.drop('contract_date', axis=1)
real_intra_day_data = real_intra_day_data.drop('contract_hour', axis=1)
real_intra_day_data = real_intra_day_data.drop('contractName', axis=1)
real_intra_day_data = real_intra_day_data.drop('prefix', axis=1)
real_intra_day_data = real_intra_day_data.drop('id', axis=1)
real_intra_day_data = real_intra_day_data[(real_intra_day_data["date"] >= "2023-04-25") & (real_intra_day_data["date"] <= "2024-04-23")]

real_intra_day_data['date'] = real_intra_day_data['date'].astype(str)
real_intra_day_data['date'] = pd.to_datetime(real_intra_day_data['date'] + ' ' + real_intra_day_data['hour'])
real_intra_day_data['date'] = pd.to_datetime(real_intra_day_data['date'])

real_intra_day_data.reset_index(drop=True, inplace=True)

real_intra_day_data


,price,quantity,date,hour
0,2522.04,150,2023-04-25 00:00:00,00:00
1,2525.01,150,2023-04-25 00:00:00,00:00
2,2600.00,3,2023-04-25 00:00:00,00:00
3,2525.02,11,2023-04-25 00:00:00,00:00
4,2525.03,5,2023-04-25 00:00:00,00:00
...,...,...,...,...
3902846,2649.00,5,2024-04-23 23:00:00,23:00
3902847,2649.00,1,2024-04-23 23:00:00,23:00
3902848,2650.00,24,2024-04-23 23:00:00,23:00
3902849,2650.00,25,2024-04-23 23:00:00,23:00


In [31]:
real_intra_day_data = real_intra_day_data.sort_values(by=['date', 'hour', 'price'])
real_intra_day_data

,price,quantity,date,hour
1894,2302.02,9,2023-04-25 00:00:00,00:00
1895,2302.02,2,2023-04-25 00:00:00,00:00
1893,2302.03,1,2023-04-25 00:00:00,00:00
1858,2357.03,2,2023-04-25 00:00:00,00:00
1862,2359.68,2,2023-04-25 00:00:00,00:00
...,...,...,...,...
3902827,2714.90,9,2024-04-23 23:00:00,23:00
3902829,2714.90,9,2024-04-23 23:00:00,23:00
3902830,2714.90,4,2024-04-23 23:00:00,23:00
3902828,2715.00,2,2024-04-23 23:00:00,23:00


In [32]:
final_intraday_df

,date,hour,price,quantile,intraday_bidded,Objective Value
0,2023-04-25 00:00:00,00:00,2522.97,0.25,0.00,1.116303e+06
1,2023-04-25 00:00:00,00:00,2535.00,0.50,0.00,1.116303e+06
2,2023-04-25 00:00:00,00:00,2580.00,0.75,0.00,1.116303e+06
3,2023-04-25 01:00:00,01:00,2522.00,0.25,0.00,1.116303e+06
4,2023-04-25 01:00:00,01:00,2529.26,0.50,0.00,1.116303e+06
...,...,...,...,...,...,...
26263,2024-04-23 22:00:00,22:00,2450.00,0.50,0.00,1.524395e+06
26264,2024-04-23 22:00:00,22:00,2595.70,0.75,53.74,1.524395e+06
26265,2024-04-23 23:00:00,23:00,2014.90,0.25,0.00,1.524395e+06
26266,2024-04-23 23:00:00,23:00,2306.00,0.50,0.00,1.524395e+06


In [33]:
for index, row in final_intraday_df.iterrows():
    if row["intraday_bidded"] == 0:
        final_intraday_df.at[index, "revenue"] = 0

    else:
        date = row["date"]
        price = row["price"]
        quantity = row["intraday_bidded"]
        threshold_quantity = real_intra_day_data[(real_intra_day_data["date"] == date) & (real_intra_day_data["price"] >= price)]["quantity"].sum()

        if threshold_quantity is None:
            final_intraday_df.at[index, "revenue"] = 0
        else:
            final_intraday_df.at[index, "revenue"] = min(threshold_quantity, quantity) * price
        

In [34]:
final_intraday_df

,date,hour,price,quantile,intraday_bidded,Objective Value,revenue
0,2023-04-25 00:00:00,00:00,2522.97,0.25,0.00,1.116303e+06,0.000
1,2023-04-25 00:00:00,00:00,2535.00,0.50,0.00,1.116303e+06,0.000
2,2023-04-25 00:00:00,00:00,2580.00,0.75,0.00,1.116303e+06,0.000
3,2023-04-25 01:00:00,01:00,2522.00,0.25,0.00,1.116303e+06,0.000
4,2023-04-25 01:00:00,01:00,2529.26,0.50,0.00,1.116303e+06,0.000
...,...,...,...,...,...,...,...
26263,2024-04-23 22:00:00,22:00,2450.00,0.50,0.00,1.524395e+06,0.000
26264,2024-04-23 22:00:00,22:00,2595.70,0.75,53.74,1.524395e+06,139492.918
26265,2024-04-23 23:00:00,23:00,2014.90,0.25,0.00,1.524395e+06,0.000
26266,2024-04-23 23:00:00,23:00,2306.00,0.50,0.00,1.524395e+06,0.000


In [35]:
dayahead_df

,date,hour_x,price,hour_y,day_ahead_bidded,Objective Value,revenue
0,2023-04-25 00:00:00,00:00,2600.00,00:00,5.46000,1.116303e+06,14196.0000
1,2023-04-25 01:00:00,01:00,2600.00,01:00,7.06000,1.116303e+06,18356.0000
2,2023-04-25 02:00:00,02:00,2235.58,02:00,7.72000,1.116303e+06,17258.6776
3,2023-04-25 03:00:00,03:00,1700.00,03:00,9.06000,1.116303e+06,15402.0000
4,2023-04-25 04:00:00,04:00,1700.00,04:00,14.02582,1.116303e+06,23843.8940
...,...,...,...,...,...,...,...
8755,2024-04-23 19:00:00,19:00,2700.00,19:00,0.00000,1.524395e+06,0.0000
8756,2024-04-23 20:00:00,20:00,2700.00,20:00,0.00000,1.524395e+06,0.0000
8757,2024-04-23 21:00:00,21:00,2700.00,21:00,0.00000,1.524395e+06,0.0000
8758,2024-04-23 22:00:00,22:00,2366.89,22:00,0.00000,1.524395e+06,0.0000


In [36]:
real_revenue = final_intraday_df['revenue'].sum() + dayahead_df['revenue'].sum()

print("Expected revenue: ", expected_total_revenue)
print("Real revenue: ", real_revenue)


Expected revenue:  455702712.64942944
Real revenue:  455702712.64942944


In [37]:
dayahead_df['date'] = pd.to_datetime(dayahead_df['date'])

dayahead_df['month'] = dayahead_df['date'].dt.month

dayahead_monthly_total_revenue = dayahead_df.groupby('month')['revenue'].sum()

print("Monthly total revenue from dayahead: ", dayahead_monthly_total_revenue)

Monthly total revenue from dayahead:  month
1     1.869508e+07
2     1.722941e+07
3     9.994265e+06
4     8.542788e+06
5     1.396210e+07
6     9.949695e+06
7     1.904821e+07
8     2.730868e+07
9     2.312350e+07
10    1.258758e+07
11    1.116959e+07
12    1.404719e+07
Name: revenue, dtype: float64


In [38]:
print("Monthly avg revenue from dayahead: ", dayahead_monthly_total_revenue.mean())
print("Total yearly revenue from dayahead: ", dayahead_monthly_total_revenue.sum())

Monthly avg revenue from dayahead:  15471507.708320588
Total yearly revenue from dayahead:  185658092.49984705


In [39]:
final_intraday_df['date'] = pd.to_datetime(final_intraday_df['date'])

final_intraday_df['month'] = final_intraday_df['date'].dt.month

intraday_monthly_total_revenue = final_intraday_df.groupby('month')['revenue'].sum()

print("Monthly total revenue from intraday: ", intraday_monthly_total_revenue)

Monthly total revenue from intraday:  month
1     3.675829e+07
2     1.688061e+07
3     1.792591e+07
4     2.953955e+07
5     1.732512e+07
6     1.655650e+07
7     3.088773e+07
8     2.011798e+07
9     2.525475e+07
10    1.750091e+07
11    2.463539e+07
12    1.666187e+07
Name: revenue, dtype: float64


In [40]:
print("Monthly avg revenue from intraday: ", intraday_monthly_total_revenue.mean())
print("Total yearly revenue from intraday: ", intraday_monthly_total_revenue.sum())

Monthly avg revenue from intraday:  22503718.345798533
Total yearly revenue from intraday:  270044620.1495824


In [41]:
total_monthly_revenue = dayahead_monthly_total_revenue + intraday_monthly_total_revenue

print("Total monthly revenue: ", total_monthly_revenue)

Total monthly revenue:  month
1     5.545337e+07
2     3.411002e+07
3     2.792018e+07
4     3.808234e+07
5     3.128722e+07
6     2.650619e+07
7     4.993594e+07
8     4.742667e+07
9     4.837825e+07
10    3.008848e+07
11    3.580498e+07
12    3.070906e+07
Name: revenue, dtype: float64


In [42]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

print("Total monthly revenue: ", total_monthly_revenue)

Total monthly revenue:  month
1    55453372
2    34110023
3    27920180
4    38082336
5    31287222
6    26506193
7    49935944
8    47426668
9    48378251
10   30088485
11   35804984
12   30709056
Name: revenue, dtype: float64


In [43]:
dayahead_bids = dayahead_df["day_ahead_bidded"].sum()
dayahead_bids

94842.03559126901

In [44]:
intraday_bids = final_intraday_df["intraday_bidded"].sum()
intraday_bids

140788.5035440457